In [1]:
import sys

sys.path.append('../GSL')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

In [3]:
import pandas as pd
import numpy as np

In [4]:
import os

In [5]:
from utils.dataset_utils import time_features_from_frequency_str
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [6]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEmbedding, self).__init__()
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        position = torch.arange(0, max_len).float().unsqueeze(1)
        div_term = (torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)).exp()

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe[:, :x.size(-1)]

In [62]:
class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model, 
                                    kernel_size=3, padding=padding, padding_mode='circular')
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight,mode='fan_in',nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x).transpose(1,2)
        return x

In [8]:
class TimeFeatureEmbedding(nn.Module):
    def __init__(self, d_model, embed_type='timeF', freq='h'):
        super(TimeFeatureEmbedding, self).__init__()

        freq_map = {'h':4, 't':5, 's':6, 'm':1, 'a':1, 'w':2, 'd':3, 'b':3}
        d_inp = freq_map[freq]
        self.embed = nn.Linear(d_inp, d_model)
    
    def forward(self, x):
        return self.embed(x)

In [9]:
num_timesteps_in = 96
num_timesteps_out = 96

In [10]:
y_df = pd.read_csv(os.path.join('../GSL/data/ETT/ETTm2/ETTm2.csv'))


y_df['date'] = pd.to_datetime(y_df['date'])

dataframe = pd.DataFrame()
time_cls = time_features_from_frequency_str('1H')
for cls_ in time_cls:
    cls_name = cls_.__class__.__name__
    dataframe[cls_name] = cls_(y_df['date'].dt)
time_stamp = dataframe.to_numpy().T


_df = y_df.drop(['date'], axis=1).to_numpy().T
X = _df.astype(np.float32)
dataset = np.expand_dims(X, axis=1)
                   
                   
indices = [
    (i, i + (num_timesteps_in + num_timesteps_out)) 
           for i in range(dataset.shape[2] - (num_timesteps_in + num_timesteps_out) + 1)
]

features, target = [], []
time_feature = []
for i, j in indices:
    features.append((dataset[:, :, i: i + num_timesteps_in]))
    target.append((dataset[:, 0, i + num_timesteps_in: j]))
    time_feature.append(time_stamp[:, i:i+num_timesteps_in])

features = torch.FloatTensor(np.array(features))
targets = torch.FloatTensor(np.array(target))
time_feature = torch.FloatTensor(np.array(time_feature))

_data = []

for batch in range(len(indices)):
    _data.append(Data(x=features[batch], y=targets[batch], time_stamp=time_feature[batch]))

In [11]:
loader = DataLoader(_data, batch_size=4, shuffle=True, drop_last=True)

In [12]:
time_featureEmbedding = TimeFeatureEmbedding(256)

In [13]:
inpt = next(iter(loader))

In [14]:
inpt.x.shape

torch.Size([28, 1, 96])

In [15]:
inpt.y.shape

torch.Size([28, 96])

In [16]:
inpt.time_stamp.shape

torch.Size([16, 96])

In [17]:
time_embed = time_featureEmbedding(inpt.time_stamp.reshape(-1, 4, 96).permute(0,2,1))

In [18]:
time_embed.shape

torch.Size([4, 96, 256])

In [19]:
position_embed = PositionalEmbedding(256)

In [20]:
pe = position_embed(inpt.x)

In [21]:
pe.shape

torch.Size([1, 96, 256])

In [63]:
token_embed = TokenEmbedding(1, 256)

In [64]:
token = token_embed(inpt.x)

In [65]:
token.shape

torch.Size([28, 96, 256])

In [25]:
(pe + token).shape

torch.Size([28, 96, 256])

In [50]:
pe.unsqueeze(dim=0).shape

torch.Size([1, 1, 96, 256])

In [51]:
token.reshape(4, 7, 96, 256).shape

torch.Size([4, 7, 96, 256])

In [54]:
time_embed.expand(7,4,96,256).permute(1,0,2,3).shape

torch.Size([4, 7, 96, 256])

In [77]:
embed_inpts = (pe.unsqueeze(dim=0) + token.reshape(4, 7, 96, 256) + 
 time_embed.expand(7, 4, 96, 256).permute(1, 0, 2, 3)).reshape(-1, 96, 256)


In [108]:
conv1 = nn.Conv1d(256, 256//4, 1)
conv2 = nn.Conv1d(256//4, 1, 1)

In [109]:
conv1.weight.shape

torch.Size([64, 256, 1])

In [97]:
embed_inpts.permute(0,2,1).shape

torch.Size([28, 256, 96])

In [93]:
o1 = conv1(embed_inpts.permute(0,2,1))

In [94]:
o1.shape

torch.Size([28, 64, 96])

In [95]:
o2 = conv2(o1)

In [96]:
o2.shape

torch.Size([28, 1, 96])

In [110]:
linear1 = nn.Linear(256, 256//4)
linear2 = nn.Linear(256//4, 1)

In [111]:
linear1.weight.shape

torch.Size([64, 256])

In [100]:
l1 = conv1(embed_inpts)

In [101]:
l1.shape

torch.Size([28, 96, 64])

In [102]:
l2 = conv2(l1)

In [103]:
l2.shape

torch.Size([28, 96, 1])

In [120]:
import re

In [124]:
temp = '15min'

In [129]:
re.split('(\d+)', temp)[-1].lower()

'min'